In [ ]:
!pip install -q autogen-agentchat[gemini] autogen-ext[openai] nest_asyncio

In [ ]:
import os, nest_asyncio
from getpass import getpass

nest_asyncio.apply()
os.environ["GEMINI_API_KEY"] = getpass("Enter your Gemini API key: ")

Enter your Gemini API key: ··········


In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=os.environ["GEMINI_API_KEY"],
    api_type="google",
)

In [ ]:
from autogen_agentchat.agents import AssistantAgent

researcher   = AssistantAgent(name="Researcher", system_message="Gather and summarize factual info.", model_client=model_client)
factchecker  = AssistantAgent(name="FactChecker", system_message="Verify facts and cite sources.",       model_client=model_client)
critic       = AssistantAgent(name="Critic",    system_message="Critique clarity and logic.",         model_client=model_client)
summarizer   = AssistantAgent(name="Summarizer",system_message="Condense into a brief executive summary.", model_client=model_client)
editor       = AssistantAgent(name="Editor",    system_message="Polish language and signal APPROVED when done.", model_client=model_client)

In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination

max_msgs = MaxMessageTermination(max_messages=20)
text_term = TextMentionTermination(text="APPROVED", sources=["Editor"])
termination = max_msgs | text_term
team = RoundRobinGroupChat(
    participants=[researcher, factchecker, critic, summarizer, editor],
    termination_condition=termination
)

In [ ]:
from autogen_agentchat.tools import TeamTool

deepdive_tool = TeamTool(team=team, name="DeepDive", description="Collaborative multi-agent deep dive")

In [ ]:
host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[deepdive_tool],
    system_message="You have access to a DeepDive tool for in-depth research."
)

In [ ]:
import asyncio

async def run_deepdive(topic: str):
    result = await host.run(task=f"Deep dive on: {topic}")
    print("🔍 DeepDive result:\n", result)
    await model_client.close()

topic = "Impacts of Model Context Protocl on Agentic AI"
loop = asyncio.get_event_loop()
loop.run_until_complete(run_deepdive(topic))

🔍 DeepDive result:
 messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Deep dive on: Impacts of Model Context Protocl on Agentic AI', type='TextMessage'), ToolCallRequestEvent(source='Host', models_usage=RequestUsage(prompt_tokens=40, completion_tokens=12), metadata={}, content=[FunctionCall(id='', arguments='{"task":"Impacts of Model Context Protocol on Agentic AI"}', name='DeepDive')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Host', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Researcher: The Model Context Protocol (MCP) aims to improve the trustworthiness and safety of AI agents by providing them with a clear understanding of their environment and the goals they are pursuing.  While the precise impacts of MCP on "agentic AI" are still being researched and developed, several potential impacts can be summarized:\n\n* **Improved Understanding & Reasoning:** MCP allows AI agents to better understand the context 